In [1]:
# !pip freeze
# would check the name & version of all libraries available for import

In [2]:
import pandas as pd
from geopy import distance

In [3]:
df = pd.read_csv('Hospitals.csv')
df_hosp = df.loc[(df.STATE=='NY')&((df.COUNTY=='NEW YORK')|(df.COUNTY== 'QUEENS')
                |(df.COUNTY=='KINGS')|(df.COUNTY=='BRONX')|(df.COUNTY=='RICHMOND'))]
df_hosp.sample(6)

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
5143,-73.960891,40.773703,7144,18110021,LENOX HILL HOSPITAL,100 E 77TH ST,NEW YORK,NY,10021,NOT AVAILABLE,...,2014-02-10T00:00:00.000Z,http://www.lenoxhillhospital.org,7002017H,NOT AVAILABLE,36,NON-PROFIT,-999,632,NOT AVAILABLE,N
2830,-73.835355,40.844184,3831,13710461,BURKE REHABILITATION CENTER,2826 WESTCHESTER AVE,BRONX,NY,10461,NOT AVAILABLE,...,2019-04-16T00:00:00.000Z,http://www.burke.org/,1046,NOT AVAILABLE,36,NON-PROFIT,-999,-999,NOT AVAILABLE,N
2391,-74.091820,40.603064,3392,120210304,STATEN ISLAND UNIV HOSP-CONCORD DIV,1050 TARGEE STREET,STATEN ISLAND,NY,10304,NOT AVAILABLE,...,2015-02-24T00:00:00.000Z,NOT AVAILABLE,7004005H,NOT AVAILABLE,36,GOVERNMENT - STATE,-999,-999,NOT AVAILABLE,N
2348,-73.998453,40.639479,3349,16811219,MAIMONIDES MEDICAL CENTER,4802 10TH AVE,BROOKLYN,NY,11219,NOT AVAILABLE,...,2014-02-10T00:00:00.000Z,http://www.maimonidesmed.org/,7001020H,NOT AVAILABLE,36,NON-PROFIT,-999,679,NOT AVAILABLE,N
5985,-73.912557,40.655269,405,16111212,BROOKDALE HOSPITAL MEDICAL CENTER,1 BROOKDALE PLAZA,BROOKLYN,NY,11212,NOT AVAILABLE,...,2014-02-10T00:00:00.000Z,http://www.brookdalehospital.org,7001002H,NOT AVAILABLE,36,NON-PROFIT,-999,530,LEVEL II,N
5139,-73.904057,40.831715,7140,15010456,BRONX-LEBANON HOSPITAL CENTER - FULTON DIVISION,1276 FULTON AVE,BRONX,NY,10456,NOT AVAILABLE,...,2014-02-10T00:00:00.000Z,http://www.bronxcare.org,7000001H,NOT AVAILABLE,36,NON-PROFIT,-999,164,NOT AVAILABLE,N


In [4]:
df_hosp.shape

(80, 34)

In [5]:
stat_df = pd.read_csv('MTAstations.csv')

In [6]:
stat_df['the_geom'].tail()

468     POINT (-73.9812359981396 40.57728100006751)
469    POINT (-74.00219709442206 40.75544635961596)
470    POINT (-73.95836178682246 40.76880251014895)
471    POINT (-73.95177090964917 40.77786104333163)
472    POINT (-73.9470660219183 40.784236650177654)
Name: the_geom, dtype: object

In [7]:
stat_coords_df = pd.DataFrame(stat_df.the_geom.str.split(' ',2).tolist(), columns = ['junk','long','lat'])

In [8]:
stat_coords_df.tail()

,junk,long,lat
468,POINT,(-73.9812359981396,40.57728100006751)
469,POINT,(-74.00219709442206,40.75544635961596)
470,POINT,(-73.95836178682246,40.76880251014895)
471,POINT,(-73.95177090964917,40.77786104333163)
472,POINT,(-73.9470660219183,40.784236650177654)


In [9]:
stat_df['long'] = pd.Series(stat_coords_df.long.str.strip('(')).astype(float)
stat_df['lat'] = pd.Series(stat_coords_df.lat.str.strip(')')).astype(float)
stat_df.tail(3)

,URL,OBJECTID,NAME,the_geom,LINE,NOTES,long,lat
470,http://web.mta.info/nyct/service/,641,72nd St,POINT (-73.95836178682246 40.76880251014895),Q,Q-all times,-73.958362,40.768803
471,http://web.mta.info/nyct/service/,642,86th St,POINT (-73.95177090964917 40.77786104333163),Q,Q-all times,-73.951771,40.777861
472,http://web.mta.info/nyct/service/,643,96th St,POINT (-73.9470660219183 40.784236650177654),Q,Q-all times,-73.947066,40.784237


In [10]:
stat_df.head()

,URL,OBJECTID,NAME,the_geom,LINE,NOTES,long,lat
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",-73.991070,40.730054
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",-74.000193,40.718803
2,http://web.mta.info/nyct/service/,3,50th St,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights",-73.983849,40.761728
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times",-73.974999,40.680862
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times",-73.894886,40.664714


In [11]:
stat_loc = pd.Series(zip(stat_df.lat, stat_df.long))
stat_loc.head()

0    (40.73005400028978, -73.99106999861966)
1    (40.71880300107709, -74.00019299927328)
2    (40.76172799961419, -73.98384899986625)
3    (40.68086213682956, -73.97499915116808)
4    (40.66471445143568, -73.89488591154061)
dtype: object

In [12]:
stat_loc_df = pd.DataFrame(stat_df['NAME'])
stat_loc_df.head()

,NAME
0,Astor Pl
1,Canal St
2,50th St
3,Bergen St
4,Pennsylvania Ave


In [13]:
stat_loc_df['lines'] = stat_df['LINE']
stat_loc_df['coords'] = stat_loc
stat_loc_df.head()

,NAME,lines,coords
0,Astor Pl,4-6-6 Express,"(40.73005400028978, -73.99106999861966)"
1,Canal St,4-6-6 Express,"(40.71880300107709, -74.00019299927328)"
2,50th St,1-2,"(40.76172799961419, -73.98384899986625)"
3,Bergen St,2-3-4,"(40.68086213682956, -73.97499915116808)"
4,Pennsylvania Ave,3-4,"(40.66471445143568, -73.89488591154061)"


Right. Haven't been doing any kind of job of commenting thus far. I have so far pulled in both the hospital and the MTA station location data and have cleaned up the latter to form ordered pairs suitable for geopy. I need to do that for the hospital data as well, then get to work writing a routine for finding the shortest distance from a station to one of the hospitals on the list.

We can either pick out the additional hospitals from Kings and Queens counties and whatever county names the Bronx and Staten Island have, or we can confine the station data to just Manhattan Island by finding some code somewhere that identifies points within a given polygon. PAG

First, let's knock out the closed hospitals.

In [14]:
still_open = (df_hosp.STATUS == 'OPEN')
still_open[5995] # The Addiction Institute of New York, which I noticed was 
                 # closed in my sample

False

In [15]:
df_hosp_open = df_hosp[still_open]
print(df_hosp.shape)
df_hosp_open.shape

(80, 34)


(72, 34)

In [16]:
df_hosp_open.head(2)

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
1153,-73.953021,40.769717,4154,18910021,GRACIE SQUARE HOSPITAL,"420 E 76TH ST, 2ND, 3RD, 4TH & 5TH FLOORS",NEW YORK,NY,10021,NOT AVAILABLE,...,2014-02-10T00:00:00.000Z,http://www.nygsh.org,1458,NOT AVAILABLE,36,NON-PROFIT,-999,157,NOT AVAILABLE,N
2346,-73.843942,40.847990,3347,15510461,CALVARY HOSPITAL,1740 EASTCHESTER RD,BRONX,NY,10461,NOT AVAILABLE,...,2014-02-10T00:00:00.000Z,http://www.calvaryhospital.org,7000011H,NOT AVAILABLE,36,NON-PROFIT,-999,225,NOT AVAILABLE,N


Now we'll zip up the coordinates for the hospital locations just like we did for station locations. The only problem is the indexing.

In [17]:
hosp_loc = pd.Series(zip(df_hosp_open.Y, df_hosp_open.X), index = df_hosp_open.index)
hosp_loc.head()

1153     (40.7697165, -73.95302135)
2346    (40.84798989, -73.84394163)
2348    (40.63947888, -73.99845263)
2350    (40.76492338, -73.95231485)
2351    (40.78492694, -73.94453023)
dtype: object

In [18]:
hosp_loc_df = pd.DataFrame(df_hosp_open['NAME'])
hosp_loc_df['address'] = df_hosp_open['ADDRESS']
hosp_loc_df['coords'] = hosp_loc

In [19]:
hosp_loc_df.head()

,NAME,address,coords
1153,GRACIE SQUARE HOSPITAL,"420 E 76TH ST, 2ND, 3RD, 4TH & 5TH FLOORS","(40.7697165, -73.95302135)"
2346,CALVARY HOSPITAL,1740 EASTCHESTER RD,"(40.84798989, -73.84394163)"
2348,MAIMONIDES MEDICAL CENTER,4802 10TH AVE,"(40.63947888, -73.99845263)"
2350,HOSPITAL FOR SPECIAL SURGERY,535 EAST 70TH ST,"(40.76492338, -73.95231485)"
2351,METROPOLITAN HOSPITAL CENTER,1901 1ST AVE,"(40.78492694, -73.94453023)"


In [20]:
hosp_loc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 1153 to 6396
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   NAME     72 non-null     object
 1   address  72 non-null     object
 2   coords   72 non-null     object
dtypes: object(3)
memory usage: 2.2+ KB


Now the task is basically to create columns in the station dataframe with the name and distance of the closest hospital to each station. Iterate over each station, find the distance to each hospital, save the name and distance of the closest hospital in the station dataframe.

In [21]:
# first let's see if we can get a single freaking distance

stat_loc_df['coords']

0      (40.73005400028978, -73.99106999861966)
1      (40.71880300107709, -74.00019299927328)
2      (40.76172799961419, -73.98384899986625)
3      (40.68086213682956, -73.97499915116808)
4      (40.66471445143568, -73.89488591154061)
                        ...                   
468     (40.57728100006751, -73.9812359981396)
469    (40.75544635961596, -74.00219709442206)
470    (40.76880251014895, -73.95836178682246)
471    (40.77786104333163, -73.95177090964917)
472    (40.784236650177654, -73.9470660219183)
Name: coords, Length: 473, dtype: object

In [22]:
stat_loc_df[0:20]

,NAME,lines,coords
0,Astor Pl,4-6-6 Express,"(40.73005400028978, -73.99106999861966)"
1,Canal St,4-6-6 Express,"(40.71880300107709, -74.00019299927328)"
2,50th St,1-2,"(40.76172799961419, -73.98384899986625)"
3,Bergen St,2-3-4,"(40.68086213682956, -73.97499915116808)"
4,Pennsylvania Ave,3-4,"(40.66471445143568, -73.89488591154061)"
5,238th St,1,"(40.88466700064975, -73.90087000018522)"
6,Cathedral Pkwy (110th St),A-B-C,"(40.800581558114956, -73.95806670661364)"
7,Kingston - Throop Aves,A-C,"(40.67991899941601, -73.94085899871263)"
8,65th St,E-M-R,"(40.74971952935675, -73.8987883783301)"
9,36th St,E-M-R,"(40.75196004401078, -73.92901818461539)"


In [23]:
stat_loc_df.iloc[0,2]

(40.73005400028978, -73.99106999861966)

In [24]:
distance.distance(stat_loc_df.iloc[0,2],hosp_loc_df.iloc[0,2]).miles

3.3877428119307282

In [28]:
# code to try the .apply(...) method to create a series of distances

def get_minimum_distance(location1, df, max_dist = 10):
  """Takes a coordinate tuple for location1, 
  a dataframe of coord tuples and location names,
  and optionally a maximum distance.
  Outputs a list containing the distance to and name of the closest location
  in the series to location1."""
  dist = max_dist
  coords = None
  place = None
  for i, location in enumerate(df['coords']):
    mi_away = distance.distance(location1, location).miles
    if mi_away < dist:
      dist = mi_away
      coords = location
      place = df.iloc[i,0]
  return [dist, place]

In [29]:
test_list = get_minimum_distance(stat_loc_df.iloc[0,2], hosp_loc_df)
test_list[1]

'NEW YORK EYE AND EAR INFIRMARY'

In [38]:
stat_hosp_df = pd.DataFrame(stat_loc_df['coords'].apply(get_minimum_distance,df=hosp_loc_df)
                           .values.tolist(), columns=['hosp_dist','hosp_name'])

In [39]:
stat_hosp_df.head(3)

,hosp_dist,hosp_name
0,0.367354,NEW YORK EYE AND EAR INFIRMARY
1,0.631470,NEW YORK-PRESBYTERIAN/LOWER MANHATTAN HOSPITAL
2,0.588702,MOUNT SINAI WEST


In [40]:
stat_hosp_full_df = pd.concat([stat_loc_df, stat_hosp_df], axis=1, sort=False)
stat_hosp_full_df.sample(10)

,NAME,lines,coords,hosp_dist,hosp_name
188,103rd St - Corona Plaza,7,"(40.749865000555545, -73.86269999830412)",1.275217,ELMHURST HOSPITAL CENTER
59,42nd St - Bryant Pk,B-D-F-M,"(40.754184001312545, -73.98459099904711)",1.007191,NYU HOSPITALS CENTER
15,Beach 105th St,A-S,"(40.58326843810286, -73.82758075034528)",4.037159,ST. JOHN'S EPISCOPAL HOSPITAL-SOUTH SHORE
11,Van Siclen Ave,J-Z,"(40.67802821447783, -73.89165772702445)",1.916232,BROOKDALE HOSPITAL MEDICAL CENTER
367,Jay St - MetroTech,A-C-F,"(40.692470636847084, -73.98721815267317)",0.507456,BROOKLYN HOSPITAL CENTER
20,174th-175th Sts,B-D,"(40.84589999983414, -73.91013600050078)",0.174365,BRONX-LEBANON HOSPITAL CENTER - CONCOURSE DIVI...
398,125th St,A-B-C-D,"(40.811071672994565, -73.95224799734774)",0.648050,MOUNT SINAI ST. LUKE'S
44,Winthrop St,2-5,"(40.65665931376077, -73.95007934590994)",0.283182,KINGS COUNTY HOSPITAL CENTER
182,Beach 98th St,A-S,"(40.58538569133279, -73.82052058959523)",3.640599,ST. JOHN'S EPISCOPAL HOSPITAL-SOUTH SHORE
94,Hunters Point Ave,7-7 Express,"(40.74221599986316, -73.9489160009391)",1.312811,NYU HOSPITALS CENTER


In [41]:
stat_hosp_full_df.columns = ['Station Name', 'Lines', 'Lat / Long', 'Hospital Distance (mi)', 'Hospital ID']
stat_hosp_full_df.tail(3)

,Station Name,Lines,Lat / Long,Hospital Distance (mi),Hospital ID
470,72nd St,Q,"(40.76880251014895, -73.95836178682246)",0.287176,GRACIE SQUARE HOSPITAL
471,86th St,Q,"(40.77786104333163, -73.95177090964917)",0.557846,LENOX HILL HOSPITAL
472,96th St,Q,"(40.784236650177654, -73.9470660219183)",0.141272,METROPOLITAN HOSPITAL CENTER


In [42]:
stat_hosp_full_df.to_csv('station_hospital_distances.csv')